In [1]:
from maelzel.snd import vamptools
from maelzel.snd import audiosample
from maelzel.snd import deverb
import pitchtools as pt
import matplotlib.pyplot as plt
import sndfileio
from math import nan

In [2]:
samples, sr = sndfileio.sndread("../snd/piano-pitchtest-Cmaj.flac")
samples2 = deverb.removeSustain(samples, sr)

In [7]:
r = vamptools.pyin(samples2, sr, overlap=8, voicedThresholdPercentile=0.05, onsetSensitivity=0.1, threshDistr='beta15', lowAmpSuppressionDb=-70, preciseTime=True, maxRelativeSkew=0.1, rmsPeriod=0.020, minRmsPercentile=0.01)

mnOut size: 1789
m_pitchTrack size: 1789


In [8]:
%matplotlib qt 
import matplotlib.pyplot as plt
axes = audiosample.Sample(samples, sr).plotSpectrogram(maxfreq=6000)
#(r.voicedProbabilityCurve * 6000).plot(axes=axes)
# r.f0curve.plot(axes=axes, linewidth=2, color='#ffff00', show=False)
c = r.smoothPitchCurveNan * (r.numCandidates > 0).nanmask()
c.plot(axes=axes, show=False, n=10000)
# r.smoothPitchCurveNan.plot(axes=axes, linewidth=2, color='#00ffff', show=False, n=10000)
# (r.smoothPitchCurve*2).plot(axes=axes, linewidth=2, color='#ffff00', show=False, n=1000)
# r.f0curve.plot(axes=axes, show=False)
# (r.voicedProbabilityCurve*10000).plot(axes=axes, show=False, n=10000)
# (r.numCandidates * 1000).plot(axes=axes, show=False, n=1000)

<Axes: >

In [9]:
parts = [b.aslinear().simplify(threshold=0.1) for b in c[::0.01].split()]
axes = plt.subplot()
for part in parts:
    print(part, len(part.points()[0]))
    part.plot(axes=axes, show=False)
plt.show()


Linear[0.02322:1.26322] 6
Linear[1.29322:2.42322] 6
Linear[2.49322:4.87322] 24
Linear[5.26322:6.58322] 4
Linear[6.61322:7.12322] 2
Linear[7.51322:8.06322] 9
Linear[8.31322:8.71322] 6
Linear[8.78322:9.03322] 5
Linear[9.09322:9.33322] 4
Linear[9.36322:9.57322] 6
Linear[9.63322:9.85322] 4
Linear[9.88322:10.3832] 11


In [10]:
import csoundengine as ce
session = ce.Session()

--Csound version 7.0 (double samples) Feb 25 2025
[commit: 2d7b783a0ed4e67b6764d69cfb29edefdaea8826]
libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [11]:
session.defInstr('vco', r'''
|kamp=0.1, kfreq=1000, iatt=0.005, irel=0.1, isust=0.7|
kgain = kfreq > 100 ? 1 : 0
kgain = lagud:k(kgain, 0.050, 0.100)
aenv = linsegr:a(0, iatt, 1, 0.010, isust, irel, 0)
outch 1, vco2:a(kamp, kfreq) * interp(kgain) * aenv
''')

def schedbpf(bpf, session):
    freqpairs = bpf.flatpairs()
    freqpairs[::2] -= bpf.x0
    synth = session.sched('vco', delay=bpf.x0, dur=bpf.x1 - bpf.x0, kfreq=freqpairs[1]) 
    synth.automate('kfreq', freqpairs, delay=bpf.x0)
    return synth

In [19]:
c = r.smoothPitchCurveNan * (r.voicedProbabilityCurve > 0.01)
c.plot(n=1000)

<Axes: >

In [16]:
import bpf4
parts = bpf4.util.split_fragments(r.f0curve)
# parts = split_fragments(r.smoothPitchCurveNan)

with session:
    for part in parts:
        s = schedbpf(part, session=session)